In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



     |████████████████████████████████| 993kB 5.0MB/s 


In [0]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as mn
from sklearn import preprocessing 

In [3]:
train_d = drive.CreateFile({'id':'12UmjkxvaYAp4_7og19z1HqwAoUN-XhYp'})
train_d.GetContentFile('train.tsv')

train = DataFrame.from_csv("train.tsv", sep="\t")
train.head(3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.


,name,item_condition_id,category_name,brand_name,price,shipping,item_description
train_id,,,,,,,
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...


In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import text, sequence
from keras.layers import Flatten, Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.regularizers import Regularizer
from keras.callbacks import TensorBoard, ModelCheckpoint

from keras import backend

Using TensorFlow backend.


In [0]:
train_n = train.shape[0]
train.price = np.log1p(train.price)
nan_feats = ['category_name', 'brand_name', 'item_description']


In [0]:
for col in nan_feats:
    train[col].fillna('missing', inplace=True)

In [0]:
le = LabelEncoder()
le.fit(np.hstack([train.category_name]))
train['category_name_labeled'] = le.transform(train.category_name)



In [0]:
le.fit(np.hstack([train.brand_name]))
train.brand_name = le.transform(train.brand_name)

In [0]:
le.fit(np.hstack([train.shipping]))
train.shipping = le.transform(train.shipping)

In [10]:
train[0:3]

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_name_labeled
train_id,,,,,,,,
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,4786,2.397895,1,No description yet,808
1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,3557,3.970292,0,This keyboard is in great condition and works ...,86
2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,4180,2.397895,1,Adorable top with a hint of lace and a key hol...,1254


In [0]:
tok = text.Tokenizer()

raw_text = np.hstack([train.name, train.item_description]) 
                      
        
tok.fit_on_texts(raw_text)

In [0]:
vocab_size = len(tok.word_index) + 1 

train.name = tok.texts_to_sequences(train.name)
train.item_description = tok.texts_to_sequences(train.item_description)

In [0]:
max_name = 10 #20
max_item_description = 75 #60


In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(train, train['price'], 
                                                      test_size=0.25, random_state=777)


In [0]:
def get_keras_dict(df):
    X = {'category_name_labeled' : np.array(df.category_name_labeled),
         'brand_name' : np.array(df.brand_name),
         'shipping' : np.array(df.shipping),
         'item_condition_id' : np.array(df.item_condition_id),            
         'name' : sequence.pad_sequences(df.name, maxlen=max_name),
         'item_description' : sequence.pad_sequences(df.item_description, maxlen=max_item_description) 
     }
    return X
   #'name' : sequence.pad_sequences(df.name, maxlen=max_name),
    #'item_description' : sequence.pad_sequences(df.item_description, maxlen=max_item_description)

In [0]:
X_train = get_keras_dict(X_train)
X_valid = get_keras_dict(X_valid)




In [0]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)

    mse = sum((y_pred-y)**2) / len(y)
    
    return np.sqrt(mse)

def eval_model(model):
    val_preds = model.predict(X_valid)
    
    y_true = np.array(y_valid.values)
    y_pred = val_preds[:, 0]
    v_rmsle = rmsle(y_true, y_pred)
    print(" RMSLE error on dev test: "+str(v_rmsle))
    return v_rmsle

In [0]:
def get_model():  
    max_category_name_labeled = np.max([train.category_name_labeled.max()])+1
    max_brand_name = np.max([train.brand_name.max()])+1
    
    category_name_labeled = Input(shape=[1], name='category_name_labeled')
    brand_name = Input(shape=[1], name='brand_name')
    shipping = Input(shape=[1], name='shipping')
    item_condition_id = Input(shape=[1], name='item_condition_id')
    
    name = Input(shape=[max_name], name='name') 
    item_description = Input(shape=[max_item_description], name='item_description')
    
    ##embeddings
    emb_category_name_labeled = Embedding(max_category_name_labeled, 10) (category_name_labeled)
    
    emb_brand_name = Embedding(max_brand_name, 10) (brand_name)
    emb_shipping = Embedding(2, 2) (shipping)
    max_item_condition_id = np.max([train.item_condition_id.max()])+1
    emb_item_condition_id = Embedding(max_item_condition_id, 5) (item_condition_id)    
    
    
    emb_name = Embedding(vocab_size+1, max_name) (name)
    emb_item_description = Embedding(vocab_size+1, max_item_description) (item_description)
    
    convs1 = []
    convs2 = []
    
    for filter_length in [1,2]:
        cnn_layer1 = Conv1D(filters=50, kernel_size=filter_length, padding='same', activation='relu', strides=1) (emb_name)
        cnn_layer2 = Conv1D(filters=50, kernel_size=filter_length, padding='same', activation='relu', strides=1) (emb_item_description)
        
        maxpool1 = GlobalMaxPooling1D() (cnn_layer1)
        maxpool2 = GlobalMaxPooling1D() (cnn_layer2)
        
        convs1.append(maxpool1)
        convs2.append(maxpool2)

    convs1 = concatenate(convs1)
    convs2 = concatenate(convs2)
    
    main_l = concatenate([
            
            Flatten() (emb_category_name_labeled),
            Flatten() (emb_brand_name),
            Flatten() (emb_shipping),
            Flatten() (emb_item_condition_id),
            
            convs1, 
            convs2, 
            
    ])
    main_l = Dropout(0.25)(Dense(128, activation='relu') (main_l)) #.25 = .435
    main_l = Dropout(0.1)(Dense(64, activation='relu') (main_l)) #.1
    
    # , kernel_regularizer=keras.regularizers.l2(0.01)
    output = Dense(1, activation='linear') (main_l)

    model = Model([category_name_labeled, brand_name, shipping, item_condition_id,
                   name, item_description], output)
    model.compile(loss='mse', optimizer='adam')
    
    return model


In [21]:
cp_fpath = 'checkpoint.hdf5' #'../output/keras-epoch_{epoch:02d}-vloss_{val_loss:.4f}.hdf5'
checkpoint_cb = ModelCheckpoint(cp_fpath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensor_cb = TensorBoard(log_dir='../graph-logs', histogram_freq=0, write_graph=True,)
#tb_cb.set_model(model)

model = get_model()


W0812 09:42:17.070554 139735019366272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 09:42:17.089517 139735019366272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 09:42:17.091545 139735019366272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 09:42:17.339112 139735019366272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0812 09:42:17.352499 

In [22]:
model.fit(X_train, y_train, 
          epochs=3, 
          batch_size=512,  
          #callbacks=[checkpoint_cb],
          validation_data=(X_valid, y_valid),
        )

W0812 09:42:29.140209 139735019366272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 1111901 samples, validate on 370634 samples
Epoch 1/3
1111901/1111901 [==============================] - 1156s 1ms/step - loss: 0.3676 - val_loss: 0.2193
Epoch 2/3
1111901/1111901 [==============================] - 1126s 1ms/step - loss: 0.2276 - val_loss: 0.2039
Epoch 3/3
1111901/1111901 [==============================] - 1159s 1ms/step - loss: 0.1926 - val_loss: 0.2009


In [23]:
v_rmsle = eval_model(model)

 RMSLE error on dev test: 0.44827366843994565


In [0]:
pred = model.predict(X_valid)

In [0]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [29]:
print("The r-squared value is: "+str(r2_score(y_valid, pred)))
print("mean absolute error:"+str(mean_absolute_error(y_valid, pred)))

The r-squared value is: 0.6427163600023145
mean absolute error:0.3357293695295104
